## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=df00187e149d3cf68d64482a9f9d1372a3fe4c7202dedea12810edb91a94413f
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import sys
import os
import shutil
# from typing_extensions import TypeAliasType

try:
    sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
except:
    sys.path.append(os.path.join(os.getcwd(), '../'))

In [ ]:
import re
import nltk
from nltk.corpus import sentiwordnet as swn
from collections import Counter
from sklearn import feature_extraction
import random
from itertools import product

In [ ]:
nltk.download('punkt')
nltk.download('sentiwordnet')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np

import pickle
from collections import Counter
from tqdm import tqdm

import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
import copy

from gensim.utils import simple_preprocess

import fasttext

np.random.seed(42)
torch.manual_seed(42)

## Load Data

In [ ]:
# train_file = '/content/drive/MyDrive/NLP_Project/IIITH_Codemixed.txt'
train_file = '/content/drive/MyDrive/nlp_project/HIT-ACL2021-Codemixed-Representation/data/hindi_sentiment/IIITH_Codemixed.txt'
df = pd.read_csv(train_file, sep='\t', header=None, usecols=[1,2])
df.columns = ['text', 'category']

## Feature Engineering for Additional Features
ie Feature Network

In [ ]:
sentences = []
with open(train_file, 'r', encoding='utf-8') as file:
    for line in file:
      parts = line.strip().split('\t')
      if len(parts) > 0:
            sentences.append(parts[1])

In [ ]:
capital_words_count = []
for sentence in sentences:
    capital_words = re.findall(r'\b[A-Z]+\b', sentence)
    capital_words_count.append(len(capital_words))

In [ ]:
extended_words_count = []
for sentence in sentences:
    extended_words = re.findall(r'\b\w*([a-zA-Z])\1\w*\b', sentence)
    # print(extended_words)
    extended_words_count.append(len(extended_words))

In [ ]:
exclamation_at_end = []
for sentence in sentences:
    exclamation_at_end.append(int(sentence.endswith('!')))

In [ ]:
repeated_punctuation_count = []

punctuation_pattern = r'(\W)\1+'

for sentence in sentences:
    repeated_punctuation = re.findall(punctuation_pattern, sentence)
    repeated_punctuation_count.append(len(repeated_punctuation))

print(repeated_punctuation_count)

[1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 2, 1, 1, 0, 5, 0, 0, 0, 0, 0, 1, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 5, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 

In [ ]:
def get_sentiment_scores(word_list):
    sentiment_scores_pos = sentiment_scores_neg = 0
    for word in word_list:
        synsets = list(swn.senti_synsets(word))
        if synsets:
            pos_score = synsets[0].pos_score()
            neg_score = synsets[0].neg_score()
            sentiment_scores_pos += pos_score
            sentiment_scores_neg += neg_score
    return sentiment_scores_pos,sentiment_scores_neg

In [ ]:
pos_sentiment_score = []
neg_sentiment_score = []

punctuation_pattern = r'(\W)\1+'

for sentence in sentences:
    pos_sentiment,negetive_sentiment = get_sentiment_scores(sentence.split(' '))
    # print(pos_sentiment)
    pos_sentiment_score.append((pos_sentiment))
    neg_sentiment_score.append((negetive_sentiment))


print(pos_sentiment_score)
print(neg_sentiment_score)

[0.0, 1.375, 0.0, 0, 0.0, 0, 1.0, 1.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.375, 0.5, 1.375, 0.0, 0.375, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.75, 0.625, 0.75, 0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.0, 0.0, 0, 0.25, 0.375, 0.0, 0.0, 0.5, 0.0, 0.0, 0.75, 0.0, 0.625, 0.0, 0.5, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0, 0.25, 0.5, 0, 0.0, 0.0, 0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.625, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.0, 0.5, 0, 0.625, 0, 0.25, 0.75, 1.125, 0, 1.125, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.5, 0.25, 0.875, 0.375, 1.0, 0.0, 0.5, 0.0, 0.5, 0.875, 0.0, 0.0, 0.0, 1.125, 0.0, 0.0, 0.25, 0.0, 0.25, 0.5, 1.0, 0.0, 0.0, 0.625, 0.0, 0.375, 0.0, 0.25, 0.0, 0.0, 0.125, 0.0, 0.0, 0, 0.0, 0.375, 0.5, 0.5, 0.0, 0.25, 0.0, 0.0, 0.0, 0, 0.875, 0.375, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.25, 0.25, 0.0, 0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.625, 0.0, 0.0, 0.

In [ ]:
df['Capital_Words_Count']= capital_words_count
df['Extended_Words_Count']= extended_words_count
df['Exclamation_at_End']= exclamation_at_end
df['Repeated_Punctuation_Count']= repeated_punctuation_count
df['Sentiment_Scores_positive']= pos_sentiment_score
df['Sentiment_Scores_negetive']= neg_sentiment_score

In [ ]:
df.head()

,text,category,Capital_Words_Count,Extended_Words_Count,Exclamation_at_End,Repeated_Punctuation_Count,Sentiment_Scores_positive,Sentiment_Scores_negetive
0,Ye song nahi hi Ye MODI Ji ka mehnat ka rang h...,Positive,1,0,0,1,0.000,0.250
1,Love u sir love u soo much urs I'ts beautyful ...,Positive,1,1,0,1,1.375,0.125
2,Arae sur jee pahelae hamare bharat ke bachho k...,Neutral,0,4,0,1,0.000,0.000
3,Wah! Jitni sundar geet ke bhao hain utnihi sun...,Positive,0,3,0,0,0.000,0.000
4,Sundar ekdam sahi Gaya Hua gana.chhotisi gudiy...,Positive,0,1,0,0,0.000,0.000


## Preprocessing

In [ ]:
def clean_tweets(text):
    text = text.lower()
    text = re.sub(r'@\w+','',text)
    text = re.sub(r'http\w+','',text)
    text = re.sub(r'#\w+','',text)
    text = re.sub(r'\d+','',text)
    text = text.strip()
    # text = text.split(' ')
    return text

In [ ]:
df.text = df.text.apply(lambda x: clean_tweets(x))

print(df[:5])

                                                text  category  \
0  ye song nahi hi ye modi ji ka mehnat ka rang h...  Positive   
1  love u sir love u soo much urs i'ts beautyful ...  Positive   
2  arae sur jee pahelae hamare bharat ke bachho k...   Neutral   
3  wah! jitni sundar geet ke bhao hain utnihi sun...  Positive   
4  sundar ekdam sahi gaya hua gana.chhotisi gudiy...  Positive   

   Capital_Words_Count  Extended_Words_Count  Exclamation_at_End  \
0                    1                     0                   0   
1                    1                     1                   0   
2                    0                     4                   0   
3                    0                     3                   0   
4                    0                     1                   0   

   Repeated_Punctuation_Count  Sentiment_Scores_positive  \
0                           1                      0.000   
1                           1                      1.375   
2             

In [ ]:
# Data Preprocessing
def add_vowel_noise(word):
    vowels = 'a'
    new_word = ''
    for char in word:
        new_word += char
        if char in vowels:
            new_word += random.choice(vowels)  # Add an extra vowel
    return new_word

def random_vowel_deletion(word):
    # Delete a vowel
    vowels = 'aeiou'
    return ''.join(char for char in word if char not in vowels or (char in vowels))

def ends_with(word):
    possible_combinations = [word]
    if len(word)>2:
        if word.endswith('yu'):
            possible_combinations.append(word + 'n')
        if word.endswith('ha'):
            possible_combinations.append(word + 'n')
        if word.endswith('haa'):
            possible_combinations.append(word + 'n')
        if word.endswith('he'):
            possible_combinations.append(word + 'y')
        if word.endswith('ne'):
            possible_combinations.append(word + 'y')
        if word.endswith('he'):
            possible_combinations.append(word + 'i')
        if word.endswith('ne'):
            possible_combinations.append(word + 'i')
        if word.endswith('oo'):
            possible_combinations.append(word[:-2] + 'u')
        if word.endswith('ee'):
            possible_combinations.append(word[:-2] + 'i')


    if len(word)>1:
        if word.endswith('u'):
            possible_combinations.append(word[:-1] + 'oo')
        if word.endswith('i'):
            possible_combinations.append(word[:-1] + 'ee')

    return possible_combinations

def introduce_noise(words):
    all_noise_words = []
    mappings = {}

    for word in tqdm(words):
        noisy_combinations = ends_with(add_vowel_noise(random_vowel_deletion(word)))
        all_noise_words.extend(noisy_combinations)
        mappings[word] = noisy_combinations

    all_noise_words = list(set(all_noise_words) - set(words))

    return all_noise_words, mappings

from collections import Counter

def extract_top_words(sentences, top_n=5000):
    # Tokenize the sentences into words
    words = ' '.join(sentences).split()

    # Use Counter to count occurrences of each word
    word_counts = Counter(words)

    # Extract the top N occurring words
    top_words = [word for word, _ in tqdm(word_counts.most_common(top_n))]

    return top_words

sentences = list(df.text)

top_words = extract_top_words(sentences, top_n=5000)

def replace_misspelled(sentences, mappings):
    reversed_mappings = {value: key for key, values in mappings.items() for value in values}
    replaced_sentences = []

    for sentence in tqdm(sentences):
        words = sentence.split()
        corrected_words = [reversed_mappings.get(word, word) if word in reversed_mappings else word for word in words]
        replaced_sentences.append(' '.join(corrected_words))

    return replaced_sentences

# sentences = list(train_df.text)

def remove_starting_space_from_list(string_list):
    return [string.lstrip() for string in string_list]

def remove_end_space_from_list(string_list):
    return [string.rstrip() for string in string_list]

# Example usage
sentences = remove_starting_space_from_list(sentences)
sentences = remove_end_space_from_list(sentences)


def percentage_difference(list1, list2):
    common_strings = set(list1).intersection(set(list2))
    total_strings = len(list1)
    different_strings = total_strings - len(common_strings)
    diff = list(set(list1) - common_strings)
    percentage_diff = (different_strings / total_strings) * 100
    return percentage_diff

misspelled_words, wmappings = introduce_noise(top_words)

# 'replaces sentences with list of sentences'
# 'if you have list of tokens, then join to get sentence'

# 'add sentences 2 to the df and tokenize again

sentences2 = replace_misspelled(sentences, wmappings)
p_result = percentage_difference(sentences, sentences2)
df.text=sentences2

100%|██████████| 3879/3879 [00:00<00:00, 304687.54it/s]


In [ ]:
def _get_unique(elems):
    if type(elems[0]) == list:
        corpus = flatten(elems)
    else:
        corpus = elems
    elems, freqs = zip(*Counter(corpus).most_common())
    return list(elems)


def convert_categorical_label_to_int(labels):
    if type(labels[0]) == list:
        uniq_labels = _get_unique(flatten(labels))
    else:
        uniq_labels = _get_unique(labels)


    if type(labels[0]) == list:
        label_to_id = {w:i+1 for i,w in enumerate(uniq_labels)}
    else:
        label_to_id = {w:i for i,w in enumerate(uniq_labels)}

    new_labels = []
    if type(labels[0]) == list:
        for i in labels:
            new_labels.append([label_to_id[j] for j in i])
    else:
        new_labels = [label_to_id[j] for j in labels]

    return new_labels, label_to_id

In [ ]:
df.category, label2idx = convert_categorical_label_to_int(df.category.values)

In [ ]:
# train_df, test_df = train_test_split(df, train_size=0.8, random_state=42)
# train_df, val_df = train_test_split(train_df, train_size=0.9, random_state=42)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(df.text):
    break

test_df = df.iloc[test_index]
kf2 = KFold(n_splits=8, shuffle=True, random_state=42)
for train_index, val_index in kf2.split(df.iloc[train_index].text):
    break

val_df = df.iloc[val_index]
train_df = df.iloc[train_index]

In [ ]:
print(len(train_df['text']), len(val_df['text']), len(test_df['text']))

2792 311 776


In [ ]:
train_df.head()

,text,category,Capital_Words_Count,Extended_Words_Count,Exclamation_at_End,Repeated_Punctuation_Count,Sentiment_Scores_positive,Sentiment_Scores_negetive
1906,ek bat bta deta hoo ap sabko.... plz ijjat se ...,Neutral,1,4,0,1,0.000,0.0
391,"bharat ""modi"" ke sath",Positive,2,1,0,2,0.000,0.0
821,rfta rfta dakho akh mari kadi h akh jisa lgi m...,Positive,0,0,0,1,0.625,0.5
526,kamal karte ho yr bhai,Neutral,0,0,0,0,0.000,0.0
2727,ek reply bhai plzzzzz,Neutral,0,1,0,1,0.000,0.0


In [ ]:
test_df.head()

,text,category,Capital_Words_Count,Extended_Words_Count,Exclamation_at_End,Repeated_Punctuation_Count,Sentiment_Scores_positive,Sentiment_Scores_negetive
1826,"hiiiii, kya yeh dream hai?",Neutral,0,1,0,0,0.000,0.0
211,modi jee ap jesa koi nhi dhanywad ap desh ki k...,Positive,0,3,0,0,1.125,0.0
2861,wtng for ur rply ....plz,Positive,0,0,0,1,0.000,0.0
410,bhai jo apno se dur ho toh unka toh yahi ek za...,Positive,0,0,0,0,0.000,0.0
1188,bhai jan nice,Positive,0,1,0,0,0.000,0.0


In [ ]:
val_df.head()

,text,category,Capital_Words_Count,Extended_Words_Count,Exclamation_at_End,Repeated_Punctuation_Count,Sentiment_Scores_positive,Sentiment_Scores_negetive
1269,bhai is bar eid ap ka chera dekh kar ho gi .ka...,Positive,0,2,0,0,0.25,0.125
1828,app ke intjar mein..luv u bhai,Positive,0,1,0,1,0.00,0.000
2530,salman bhai...suna hai ap ko kamar k takleef h...,Neutral,4,4,0,3,0.00,0.000
3231,ok salman bha!!!!,Neutral,0,0,1,1,0.00,0.000
1294,zara pucho salman se k kitna raha,Neutral,0,0,0,0,0.00,0.000


In [ ]:
idx2label = {i:w for (w, i) in label2idx.items()}
print(idx2label)

{0: 'Neutral', 1: 'Positive', 2: 'Negative'}


## Pretrained SentencePiece Embeddings

In [ ]:
vocab,embeddings = [],[]
with open('/content/drive/MyDrive/nlp_project/custom_embedding.txt','rt') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    embeddings.append(i_embeddings)

In [ ]:
vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)

In [ ]:
len(vocab_npa), print(vocab_npa[:10])

['<unk>' '<s>' '</s>' 's' '▁the' '▁' '▁to' 'e' '▁i' '▁you']


(7000, None)

In [ ]:
embs_npa.shape

(7000, 128)

In [ ]:
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
# vocab_npa = np.insert(vocab_npa, 1, '[UNK]')
print(vocab_npa[:10])

pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
# unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

#insert embeddings for pad and unk tokens at top of embs_npa.
# embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
embs_npa = np.vstack((pad_emb_npa,embs_npa))
print(embs_npa.shape)

['<pad>' '<unk>' '<s>' '</s>' 's' '▁the' '▁' '▁to' 'e' '▁i']
(7001, 128)


## Training Embeddings Using FastText

In [ ]:
df.text.to_csv(r'data.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
fastText_model = fasttext.train_unsupervised('data.txt', model='skipgram', dim=300, minn=2, maxn=10)

In [ ]:
fastText_model['the'].shape

(300,)

In [ ]:
fastText_model.get_input_matrix().shape

(2002009, 300)

In [ ]:
# Preprocessing for Simple BiLSTM
def text_to_sequence(text, fastText_model, max_seq_len=40, embedding_size=300):
    tokens = simple_preprocess(text)
    vectors = []
    for token in text:
        if token in fastText_model:
            vectors.append(fastText_model[token])
        else:
            vectors.append(np.zeros(embedding_size))

    if len(vectors) < max_seq_len:
        padding = [np.zeros(embedding_size)] * (max_seq_len - len(vectors))
        vectors += padding
    else:
        vectors = vectors[:max_seq_len]

    emb = np.array(vectors)
    return emb

def generate_subword_embeddings(df, fastText_model, max_seq_len=40):
    embeddings = []

    for text in df['text']:
        emb = text_to_sequence(text, fastText_model, max_seq_len)
        embeddings.append(emb)

    return np.array(embeddings),np.array(df.drop(['text', 'category'],axis=1))

In [ ]:
word2idxLower = {
    word: index
    for index, word in enumerate(vocab_npa)
}

In [ ]:
train_embeddings, train_fn = generate_subword_embeddings(train_df, fastText_model)
val_embeddings, val_fn = generate_subword_embeddings(val_df, fastText_model)
test_embeddings, test_fn = generate_subword_embeddings(test_df, fastText_model)

In [ ]:
# Preprocessing for Modified Mukherjee
def text_to_sequence(text, fastText_model, max_seq_len=40, embedding_size=300):
    tokens = simple_preprocess(text)
    vectors = []
    # Fasttext
    for token in text:
        if token in fastText_model:
            vectors.append(fastText_model[token])
        else:
            vectors.append(np.zeros(embedding_size))

    if len(vectors) < max_seq_len:
        padding = [np.zeros(embedding_size)] * (max_seq_len - len(vectors))
        vectors += padding
    else:
        vectors = vectors[:max_seq_len]

    emb = np.array(vectors)
    # sentencepiece
    seq=  [
        word2idxLower.get(word, word2idxLower.get('<unk>'))
        for word in text
    ]
    if len(seq) < max_seq_len:
        padding = [0] * (max_seq_len - len(seq))
        seq += padding
    else:
        seq = seq[:max_seq_len]
    return emb, seq

def generate_subword_embeddings(df, fastText_model, max_seq_len=40):
    embeddings = []
    embeddings_bng = []
    for text in df['text']:
        emb, emb_bng = text_to_sequence(text, fastText_model, max_seq_len)
        embeddings.append(emb)
        embeddings_bng.append(emb_bng)


    return np.array(embeddings),np.array(embeddings_bng), np.array(df.drop(['text', 'category'],axis=1))

In [ ]:
train_embeddings,train_embeddings_bng, train_fn = generate_subword_embeddings(train_df, fastText_model)
val_embeddings,val_embeddings_bng, val_fn = generate_subword_embeddings(val_df, fastText_model)
test_embeddings ,test_embeddings_bng, test_fn = generate_subword_embeddings(test_df, fastText_model)

In [ ]:
train_embeddings = torch.Tensor(train_embeddings)
train_embeddings_bng = torch.Tensor(train_embeddings_bng)
train_fn = torch.Tensor(train_fn)
train_labels = torch.Tensor(train_df['category'].values)

val_embeddings = torch.Tensor(val_embeddings)
val_embeddings_bng = torch.Tensor(val_embeddings_bng)
val_fn = torch.Tensor(val_fn)
val_labels = torch.Tensor(val_df['category'].values)

test_embeddings = torch.Tensor(test_embeddings)
test_embeddings_bng = torch.Tensor(test_embeddings_bng)
test_fn = torch.Tensor(test_fn)
test_labels = torch.Tensor(test_df['category'].values)


In [ ]:
batch_size = 64
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_data = TensorDataset(train_embeddings, train_embeddings_bng , train_fn, train_labels)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = TensorDataset(val_embeddings, val_embeddings_bng , val_fn, val_labels)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

test_data = TensorDataset(test_embeddings, test_embeddings_bng , test_fn, test_labels)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
def maori_train_with_early_stopping(model, train_loader=train_loader, val_loader=val_loader, num_epochs=50, patience=10):
    best_val_loss = float('inf')
    best_val_f1 = 0
    current_patience = 0
    best_model = None

    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_loss = 0

        for inputs, inputs_fn, labels in train_loader:
            inputs, inputs_fn, labels = inputs.to(device), inputs_fn.to(device), labels.to(device)
            optimizer.zero_grad()

            out = model(inputs, inputs_fn)
            loss = criterion(out, labels.long())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        mean_loss = total_loss / len(train_loader)

        model.eval()
        val_loss = 0
        val_predictions = []
        val_targets = []
        correct, total = 0, 0

        with torch.no_grad():
            for inputs,inputs_fn, labels in val_loader:
                inputs, inputs_fn, labels = inputs.to(device), inputs_fn.to(device), labels.to(device)
                out = model(inputs, inputs_fn)

                loss = criterion(out, labels.long())
                val_loss += loss.item()

                _, predicted = torch.max(out, 1)

                val_predictions.extend(predicted.cpu().long().numpy())
                val_targets.extend(labels.cpu().numpy())

                # print(len(val_targets), val_targets[-10:])
                # print(len(val_predictions), val_predictions[-10:])

        # val_accuracy = accuracy_score(val_targets, val_predictions)
        val_loss /= len(val_loader)
        val_w_f1 = f1_score(val_targets, val_predictions, average='weighted')

        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {mean_loss}, Validation Loss: {val_loss}, Validation F1: {val_w_f1}')

        if val_w_f1 > best_val_f1:
            best_val_f1 = val_w_f1
            current_patience = 0
            best_model = copy.deepcopy(model)
        else:
            current_patience += 1

        # if current_patience >= patience:
        #     print(f'Stopping after {epoch+1} epochs')
        #     break

    return best_model

In [ ]:
def maori_test_metrics(model, test_loader):
    model.eval()
    test_predictions = []
    test_targets = []
    with torch.no_grad():
        for inputs, inputs_fn, labels in test_loader:
            inputs, inputs_fn, labels = inputs.to(device), inputs_fn.to(device), labels.to(device)
            out = model(inputs, inputs_fn)

            _, predicted = torch.max(out, 1)
            test_predictions.extend(predicted.cpu().long().numpy())
            test_targets.extend(labels.cpu().numpy())

    print(len(test_targets), len(test_predictions))

    test_acc = accuracy_score(test_targets, test_predictions)
    test_w_f1 = f1_score(test_targets, test_predictions, average='weighted')
    test_macro_f1 = f1_score(test_targets, test_predictions, average='macro')
    class_report = classification_report(test_targets, test_predictions)

    print('Accuracy:', test_acc)
    print('F1-Weighted:', test_w_f1)
    print('F1-Macro:', test_macro_f1)
    print('Classification Report:', class_report, sep='\n')

In [ ]:
def train_with_early_stopping(model, train_loader=train_loader, val_loader=val_loader, num_epochs=50, patience=10):
    best_val_loss = float('inf')
    best_val_f1 = 0
    current_patience = 0
    best_model = None

    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_loss = 0

        for inputs, inputs_bng, inputs_fn, labels in train_loader:
            inputs,inputs_bng, inputs_fn, labels = inputs.to(device),inputs_bng.to(device), inputs_fn.to(device), labels.to(device)
            optimizer.zero_grad()

            out = model(inputs,inputs_bng, inputs_fn)
            loss = criterion(out, labels.long())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        mean_loss = total_loss / len(train_loader)

        model.eval()
        val_loss = 0
        val_predictions = []
        val_targets = []
        correct, total = 0, 0

        with torch.no_grad():
            for inputs,inputs_bng, inputs_fn, labels in val_loader:
                inputs,inputs_bng, inputs_fn, labels = inputs.to(device),inputs_bng.to(device), inputs_fn.to(device), labels.to(device)
                out = model(inputs,inputs_bng, inputs_fn)

                loss = criterion(out, labels.long())
                val_loss += loss.item()

                _, predicted = torch.max(out, 1)

                val_predictions.extend(predicted.cpu().long().numpy())
                val_targets.extend(labels.cpu().numpy())

                # print(len(val_targets), val_targets[-10:])
                # print(len(val_predictions), val_predictions[-10:])

        # val_accuracy = accuracy_score(val_targets, val_predictions)
        val_loss /= len(val_loader)
        val_w_f1 = f1_score(val_targets, val_predictions, average='weighted')

        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {mean_loss}, Validation Loss: {val_loss}, Validation F1: {val_w_f1}')

        if val_w_f1 > best_val_f1:
            best_val_f1 = val_w_f1
            current_patience = 0
            best_model = copy.deepcopy(model)
        else:
            current_patience += 1

        if current_patience >= patience:
            print(f'Stopping after {epoch+1} epochs')
            break

    return best_model

In [ ]:
def test_metrics(model, test_loader):
    model.eval()
    test_predictions = []
    test_targets = []
    with torch.no_grad():
        for inputs,inputs_bng, inputs_fn, labels in test_loader:
            inputs,inputs_bng, inputs_fn, labels = inputs.to(device),inputs_bng.to(device), inputs_fn.to(device), labels.to(device)
            out = model(inputs,inputs_bng, inputs_fn)

            _, predicted = torch.max(out, 1)
            test_predictions.extend(predicted.cpu().long().numpy())
            test_targets.extend(labels.cpu().numpy())

    print(len(test_targets), len(test_predictions))

    test_acc = accuracy_score(test_targets, test_predictions)
    test_w_f1 = f1_score(test_targets, test_predictions, average='weighted')
    test_macro_f1 = f1_score(test_targets, test_predictions, average='macro')
    class_report = classification_report(test_targets, test_predictions)

    print('Accuracy:', test_acc)
    print('F1-Weighted:', test_w_f1)
    print('F1-Macro:', test_macro_f1)
    print('Classification Report:', class_report, sep='\n')

## Simple BiLSTM Model

In [ ]:
class Maori(nn.Module): #Ignore
    def __init__(self, input_size=300, hidden_size=128, out_size=3, dropout_prob=0.5):
        super(Maori, self).__init__()
        # self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())
        # self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float(), freeze=True)
        self.lstm = nn.LSTM(input_size, 128, batch_first=True, bidirectional=True)
        self.drop = nn.Dropout(dropout_prob)
        self.linear1 = nn.Linear(2*128, 128)
        self.linear2 = nn.Linear(128, out_size)
        self.relu = nn.ReLU()

    def forward(self, x, fn):
        out = self.drop(x)
        out, _ = self.lstm(x)
        # out = self.drop(out)


        # out = torch.cat((out[:, -1, :], fn), dim=1)
        # out = self.relu(self.linear1(out))
        out = self.relu(self.linear1(out[:, -1, :]))
        out = self.relu(self.linear2(out))

        return out

model = Maori(out_size=len(idx2label), dropout_prob=0.)
model = model.to(device)
print(model)

Maori(
  (lstm): LSTM(300, 128, batch_first=True, bidirectional=True)
  (drop): Dropout(p=0.0, inplace=False)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3, bias=True)
  (relu): ReLU()
)


In [ ]:
# class_weights = [1.0,5.0,5.0]
# criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters())

In [ ]:
maori_train_with_early_stopping(model, num_epochs=50)

  2%|▏         | 1/50 [00:00<00:19,  2.58it/s]

Epoch [1/50], Train Loss: 1.0126256102865392, Validation Loss: 1.0252678394317627, Validation F1: 0.31738143956793474


  4%|▍         | 2/50 [00:00<00:15,  3.11it/s]

Epoch [2/50], Train Loss: 0.9786363677545027, Validation Loss: 1.0160928606986999, Validation F1: 0.31738143956793474


  6%|▌         | 3/50 [00:00<00:13,  3.37it/s]

...

 98%|█████████▊| 49/50 [00:13<00:00,  3.21it/s]

Epoch [49/50], Train Loss: 0.9420308077877219, Validation Loss: 0.9962305307388306, Validation F1: 0.4715268208033481


100%|██████████| 50/50 [00:14<00:00,  3.53it/s]

Epoch [50/50], Train Loss: 0.9432235712354834, Validation Loss: 0.9898252367973328, Validation F1: 0.44665764088678284


Maori(
  (lstm): LSTM(300, 128, batch_first=True, bidirectional=True)
  (drop): Dropout(p=0.0, inplace=False)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3, bias=True)
  (relu): ReLU()
)

In [ ]:
maori_test_metrics(model, val_loader)

311 311
Accuracy: 0.5080385852090032
F1-Weighted: 0.44665764088678284
F1-Macro: 0.34561403508771926
Classification Report:
              precision    recall  f1-score   support

         0.0       0.52      0.79      0.63       151
         1.0       0.48      0.36      0.41       108
         2.0       0.00      0.00      0.00        52

    accuracy                           0.51       311
   macro avg       0.33      0.38      0.35       311
weighted avg       0.42      0.51      0.45       311



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
maori_test_metrics(model, test_loader)

776 776
Accuracy: 0.5154639175257731
F1-Weighted: 0.46532242121602624
F1-Macro: 0.35014204545454547
Classification Report:
              precision    recall  f1-score   support

         0.0       0.56      0.74      0.64       395
         1.0       0.43      0.40      0.41       265
         2.0       0.00      0.00      0.00       116

    accuracy                           0.52       776
   macro avg       0.33      0.38      0.35       776
weighted avg       0.43      0.52      0.47       776



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modified Mukherjee Model
SentencePiece+FastText Embeddings with Attention and Feature Network

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted


In [ ]:
class MukherjeeV3(nn.Module):
    def __init__(self, config):
        super(MukherjeeV3, self).__init__()

        # Word Embeddings
        self.lstm_word = nn.LSTM(config['word_input_size'], config['lstm_word_hidden'], num_layers=2, batch_first=True, bidirectional=True)
        # self.dropout_word = nn.Dropout(config["drop_word"])
        self.attention_word = SelfAttention(config['lstm_word_hidden']*2)
        # self.multihead_attn_word = nn.MultiheadAttention(config['lstm_word_hidden']*2, config['num_heads'])

        # Char Embeddings
        # self.lstm_char = nn.LSTM(config['input_size'], config['lstm_char_hidden'], batch_first=True, bidirectional=True)
        # self.dropout_char = nn.Dropout(config["drop_char"])

        #Bangla Embeddings
        self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float(), freeze=True)
        self.lstm = nn.LSTM(input_size = config['subword_input_size'], hidden_size = config['lstm_subword_hidden'], num_layers=4, bidirectional=True, batch_first=True)
        # self.linear = nn.Linear(config['hidden_dim'] * 2,  config['num_heads'])
        # self.dropout = nn.Dropout(config['drop_char'])
        self.attention_subword = SelfAttention(config['lstm_subword_hidden']*2)
        # self.multihead_attn_subword = nn.MultiheadAttention(config['hidden_dim']*2, config['num_heads'])

        # Dense Layers
        self.attention_both = SelfAttention(config['lstm_word_hidden']*2+config['lstm_subword_hidden']*2)
        # self.multihead_attn_both = nn.MultiheadAttention(config['lstm_word_hidden']*2+config['hidden_dim']*2, config['num_heads'])

        # self.dropout_fc1 = nn.Dropout(config["drop_fc"])
        # self.dropout_fc2 = nn.Dropout(config["drop_fc"])
        self.fc1 = nn.Linear(config['fc1_dim'], config['fc2_dim'])
        self.fc2 = nn.Linear(config['fc2_dim'], config['fc3_dim'])
        self.fc3 = nn.Linear(config['fc3_dim'], config['n_classes'])
        self.relu = nn.ReLU()

    def forward(self, x, x_bng, fn):
        # Word Embeddings
        out, (hidden,_) = self.lstm_word(x)
        # out = self.dropout_word(out)
        out_word = self.attention_word(out)
        # out_word, _ = self.multihead_attn_word(out, out, out)

        # Char Embeddings
        # out, (hidden,_) = self.lstm_char(x)
        # out_char = self.dropout_char(out)
        out_subword = self.embedding(x_bng.long())
        # print(x[0])
        # print(embedded[0])
        # print(embedded.shape)
        out_subword, (hidden, _) = self.lstm(out_subword)
        # embedded = self.dropout(embedded)
        out_subword = self.attention_subword(out_subword)
        # out, _ = self.multihead_attn_subword(embedded, embedded, embedded)
        # outputs = self.linear(outputs)
        # embedded =  outputs.transpose(1, 2)

        # print(x[0])
        # print(embedded[0])



        # Concatenate Word and Char Embeddings
        # print(out_word.shape)
        # print(embedded.shape)
        out = torch.cat((out_word, out_subword), dim=2)
        # print(out.shape)
        # print(out[:, -1, :].shape)
        # expected_input_size = config['lstm_word_hidden']*2 + config['hidden_dim']*2
        # print(f"Expected input size of fully connected layer: {expected_input_size}")
        # out = out_word

        # Attention
        out = self.attention_both(out)
        # out = self.multihead_attn_both(out,out,out)
        # print(out.shape)
        # print(out[:, -1, :].shape)
        # print(fn.shape)
        # out = torch.cat((out[:, -1, :], fn), dim=1)
        # print(out.shape)
        # x = torch.cat((x, any_caps.unsqueeze(2)), dim=2)
        # Dense Layers
        # out = self.relu(self.fc1(out))
        out = self.relu(self.fc1(out[:, -1, :]))
        # out = self.dropout_fc1(out)
        out = self.relu(self.fc2(out))
        # out = self.dropout_fc2(out)
        out = self.relu(self.fc3(out))
        return out

In [ ]:
config={'word_input_size':300,
        'subword_input_size':128,
        'lstm_word_hidden':300,
        'lstm_subword_hidden':128,
        # 'lstm_char_hidden':128,
        'fc1_dim':300*2+128*2,
        'fc2_dim':512,
        'fc3_dim':128,
        'n_classes':len(idx2label),
        'drop_subword':0.2,
        'drop_word':0.4,
        'drop_fc':0.4,
        # 'num_heads':8,
        # 'embedding_dim' :128,
        # 'hidden_dim' : 256,
        }

model = MukherjeeV3(config)
model = model.to(device)
print(model)

MukherjeeV1(
  (lstm_word): LSTM(300, 300, num_layers=2, batch_first=True, bidirectional=True)
  (attention_word): SelfAttention(
    (query): Linear(in_features=600, out_features=600, bias=True)
    (key): Linear(in_features=600, out_features=600, bias=True)
    (value): Linear(in_features=600, out_features=600, bias=True)
    (softmax): Softmax(dim=2)
  )
  (embedding): Embedding(7001, 128)
  (lstm): LSTM(128, 128, num_layers=4, batch_first=True, bidirectional=True)
  (attention_subword): SelfAttention(
    (query): Linear(in_features=256, out_features=256, bias=True)
    (key): Linear(in_features=256, out_features=256, bias=True)
    (value): Linear(in_features=256, out_features=256, bias=True)
    (softmax): Softmax(dim=2)
  )
  (attention_both): SelfAttention(
    (query): Linear(in_features=856, out_features=856, bias=True)
    (key): Linear(in_features=856, out_features=856, bias=True)
    (value): Linear(in_features=856, out_features=856, bias=True)
    (softmax): Softmax(dim=2

In [ ]:
# class_weights = [1.0,5.0,5.0]
# criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters())

In [ ]:
best_model = train_with_early_stopping(model, num_epochs=50)

  2%|▏         | 1/50 [00:02<02:11,  2.68s/it]

Epoch [1/50], Train Loss: 0.9999884556640278, Validation Loss: 1.0290299892425536, Validation F1: 0.31738143956793474


  4%|▍         | 2/50 [00:05<02:05,  2.62s/it]

Epoch [2/50], Train Loss: 0.9449486637657339, Validation Loss: 0.9782434940338135, Validation F1: 0.37363538101682436


  6%|▌         | 3/50 [00:07<02:02,  2.60s/it]

...

Epoch [20/50], Train Loss: 0.14913088345730846, Validation Loss: 1.8600791215896606, Validation F1: 0.6331871032236824


 42%|████▏     | 21/50 [00:55<01:18,  2.70s/it]

Epoch [21/50], Train Loss: 0.1583293251354586, Validation Loss: 2.1493008136749268, Validation F1: 0.622958749003765


 42%|████▏     | 21/50 [00:57<01:20,  2.76s/it]

Epoch [22/50], Train Loss: 0.13371431996876543, Validation Loss: 1.9396701335906983, Validation F1: 0.6497668745956018
Stopping after 22 epochs


In [ ]:
test_metrics(best_model, test_loader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


776 776
Accuracy: 0.6108247422680413
F1-Weighted: 0.6091328774086275
F1-Macro: 0.5667373848718545
Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.66      0.67       395
         1.0       0.55      0.64      0.59       265
         2.0       0.50      0.39      0.44       116

    accuracy                           0.61       776
   macro avg       0.58      0.56      0.57       776
weighted avg       0.61      0.61      0.61       776



In [ ]:
test_metrics(best_model, val_loader)

388 388
Accuracy: 0.6262886597938144
F1-Weighted: 0.5786135747404311
F1-Macro: 0.44540006863599163
Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.75      0.72       198
         1.0       0.55      0.71      0.62       133
         2.0       0.00      0.00      0.00        57

    accuracy                           0.63       388
   macro avg       0.41      0.49      0.45       388
weighted avg       0.54      0.63      0.58       388



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

In [ ]:
test_metrics(best_model, train_loader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


2715 2715
Accuracy: 0.7653775322283609
F1-Weighted: 0.7116332715285199
F1-Macro: 0.5390638523981881
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.92      0.86      1457
         1.0       0.71      0.83      0.76       900
         2.0       0.00      0.00      0.00       358

    accuracy                           0.77      2715
   macro avg       0.50      0.58      0.54      2715
weighted avg       0.66      0.77      0.71      2715



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
torch.save(best_model.state_dict(), '/content/drive/MyDrive/nlp_project/MukherjeeV2SpellCheck.pt')